In [ ]:
%matplotlib inline
import numpy as np
import torch
import scipy
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm

from scipy.stats import levy_stable
from scipy.special import gamma as gamma_func

In [ ]:
class Fractional_Langevin_MonteCarlo():
    def __init__(self, m=None):
        self.m = m

    def phi(self, U):
        # unnormalized target density : exp(-U(X))
        return torch.exp(-U)

    def drift_hat(self, x, alpha, U):
        # simplified multidimensional drift
        # approximate fractional derivative by using only the first term of centered difference operator

        x.requires_grad_()
        u = U(x)
        grad = torch.autograd.grad(u, x)[0]
        
        def c_alpha(alpha):
            return gamma_func(alpha-1) / gamma_func(alpha/2)**2

        return -c_alpha(alpha) * grad
    
    def FLA(self, U, alpha, N, a_eta, b_eta, step_size, dim):
        # Fractional Langevin Algorithm(FLA)
        # when alpha = 2, FLA is same as ULA
        burn_in = 5000
        x_0 = torch.randn(1, dim).double()

        Levy_motions = levy_stable.rvs(alpha, 0, size=dim * (N + burn_in))
        Levy_motions = torch.from_numpy(Levy_motions.reshape((Levy_motions.shape[0] // dim, dim)))
        #Brownian_motions = torch.randn(N + burn_in, dim)

        x_i = x_0
        samples = []
        num_nan = 0

        for i in tqdm.tqdm(range(N + burn_in)):
            #step_size = (a_eta/(i+1))**b_eta
            step_size = step_size
            b = self.drift_hat(x_i, alpha, U)
            x_i = x_i.detach() + step_size * b + step_size ** (1./alpha) * Levy_motions[i]
            #x_i = x_i.detach() + step_size * b + step_size ** (1./alpha) * Brownian_motions[i]                        

            if True in np.isnan(x_i):
                num_nan += 1
            else:
                if dim > 1:
                    samples.append(x_i.detach().numpy())
                else:
                    samples.append(x_i.detach().numpy().squeeze())

        print(f"{N} samples are generated, and then {num_nan} samples are excluded due to NAN")
        if dim > 1:
            return np.concatenate(samples, 0)[burn_in:]
        else:
            return samples[burn_in:]

FLMC = Fractional_Langevin_MonteCarlo()

# Experiment #1
1 dimensional space

In [ ]:
FLMC = Fractional_Langevin_MonteCarlo()

In [ ]:
# Section 4. Experiment with double-well potential
def double_well_potential_np(x):
    U = (x+5)*(x+1) * (x-1.02) * (x-5) / 10 + 0.5
    return np.exp(-U)

def double_well_potential(x):
    U = (x+5)*(x+1) * (x-1.02) * (x-5) / 10 + 0.5
    return U

In [ ]:
# Visualize Figure 2 (top) in paper
x = np.linspace(-6, 6, 1000)
y = double_well_potential_np(x)

plt.figure(figsize = (10,2))
plt.plot(x, y)
plt.title('Figure 2 (Top)')
plt.show()

In [ ]:
# Visualize Figure 2 (middle) in paper
# empirical distribution obtained via ULA (corresponds to FLA with alpha = 2)
samples = FLMC.FLA(U=double_well_potential, alpha=2, N=100000, a_eta=1e-7, b_eta=0.6, step_size=0.001, dim=1)

plt.figure(figsize = (10,2))
plt.hist(samples, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Figure 2 (Middle)')
plt.show()

In [ ]:
# Visualize Figure 2 (bottom) in paper
# empirical distribution obtained via FLA (alpha = 1.7)
samples = FLMC.FLA(U=double_well_potential, alpha=1.75, N=100000, a_eta=1e-7, b_eta=0.6, step_size=0.001, dim=1)

plt.figure(figsize = (10,2))
plt.hist(samples, bins=200, density=True)
plt.title('Figure 2 (Middle)')
plt.show()

# Experiment #2
2 dimensional space

In [ ]:
def potential1_np(z):
    z1, z2 = z[:, 0], z[:, 1]
    norm = np.sqrt(z1 ** 2 + z2 ** 2)
    exp1 = np.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = np.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - np.log(exp1 + exp2)
    return np.exp(-u)

def potential1(z):
    z = z.view(-1, 2).double()
    z1, z2 = z[:, 0], z[:, 1]
    norm = torch.norm(z, p=2, dim=1)
    exp1 = torch.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = torch.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - torch.log(exp1 + exp2)
    return u

In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = potential1_np(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Density #1')
plt.show()

In [ ]:
samples1 = FLMC.FLA(potential1, alpha=1.7, N =100000, a_eta=1e-7, b_eta=0.6, step_size=0.00001, dim=2)

plt.hist2d(samples1[:,0], samples1[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #1')
plt.show()

In [ ]:
def potential2_np(z):
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return np.exp(-u)


def potential2(z):
    z = z.view(-1, 2).double()
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return u

In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = potential2_np(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Density #2')
plt.show()

In [ ]:
samples2 = FLMC.FLA(potential2, alpha=1.7, N =100000, a_eta=1e-7, b_eta=0.6, step_size=0.001, dim=2)

plt.hist2d(samples2[:,0], samples2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #2')
plt.show()

# Experiment #3
2 dimensional space with more complicated multi-modal density

In [ ]:
def multivariate_normal_np(x, mean, cov, d):
    """pdf of the multivariate normal distribution."""
    # 2-dimension
    B = x.shape[0]
    mean = np.array([[mean[0], mean[1]]])
    
    x_m = x - mean
    x_m = np.array(x_m).reshape(B, d, 1)

    first_term = 1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(cov)))
    temp = np.linalg.solve(cov, x_m)
    temp = temp.reshape(B,1,d)
    u = np.matmul(temp,x_m) / 2
    second_term = np.exp(-u)

    out = (first_term * second_term).squeeze()

    return out

    
def gaussian_mixture_np(z):
    # d : dimension of x - Multivariate Gaussian
    # m : number of mixtures - Gaussian Mixture
    # means : m x d
    d = 2
    m = 6
    means = np.array([[-2,0.5],[-1.5,-1],[0.5,-2],[2,-0.5],[1.5,1],[-1,1.5]])
    covs = np.array([[[0.5,0],[0,1]],[[1,-0.5],[-0.5,1]],[[1,0.5],[0.5,1]],[[0.5,0],[0,1]],[[1,-0.5],[-0.5,1]],[[1,0],[0,0.5]]])

    u = np.zeros(z.shape[0], dtype='float64')

    for i in range(m):
        u += multivariate_normal_np(z, means[i], covs[i], d)

    return u

def multivariate_normal(x, mean, cov, d):
    """pdf of the multivariate normal distribution."""
    # 2-dimension    
    x_m = (x - mean).T

    first_term = 1. / (torch.sqrt((2 * torch.pi)**d * torch.linalg.det(cov)))
    temp = torch.linalg.solve(cov, x_m)
    u = torch.matmul(temp.T, x_m) / 2
    u *= first_term

    return u.squeeze(-1)

    
def gaussian_mixture(z):
    # d : dimension of x - Multivariate Gaussian
    # m : number of mixtures - Gaussian Mixture
    # means : m x d
    d = 2
    m = 6
    means = torch.tensor([[-2,0.5],[-1.5,-1],[0.5,-2],[2,-0.5],[1.5,1],[-1,1.5]], dtype=torch.double, requires_grad=True)
    covs = torch.tensor([[[0.5,0],[0,1]],[[1,-0.5],[-0.5,1]],[[1,0.5],[0.5,1]],[[0.5,0],[0,1]],[[1,-0.5],[-0.5,1]],[[1,0],[0,0.5]]], dtype=torch.double, requires_grad=True)

    z = z.view(-1, 2).double()

    u = 0
    for i in range(m):
        u += multivariate_normal(z, means[i], covs[i], d)

    return u

r = np.linspace(-3, 3, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = gaussian_mixture_np(z)
q0 = np.array(q0)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Gaussian mixture')
plt.show()

In [ ]:
samples3 = FLMC.FLA(gaussian_mixture, alpha=2.0, N =100000, a_eta=1e-7, b_eta=0.6, step_size=0.1, dim=2)

plt.hist2d(samples3[:,0], samples3[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #3')
plt.show()

In [ ]:
def multivariate_normal_np(x, mean, cov, d):
    """pdf of the multivariate normal distribution."""
    # 2-dimension
    B = x.shape[0]
    mean = np.array([[mean[0], mean[1]]])
    
    x_m = x - mean
    x_m = np.array(x_m).reshape(B, d, 1)

    first_term = 1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(cov)))
    temp = np.linalg.solve(cov, x_m)
    temp = temp.reshape(B,1,d)
    u = np.matmul(temp,x_m) / 2
    second_term = np.exp(-u)

    out = (first_term * second_term).squeeze()

    return out

    
def gaussian_mixture_np(z):
    # d : dimension of x - Multivariate Gaussian
    # m : number of mixtures - Gaussian Mixture
    # means : m x d
    d = 2
    m = 2
    means = np.array([[-2,0.5],[0.5,-2],[1.5,1]])
    covs = np.array([[[0.5,0],[0,1]],[[0.5,0],[0,1]],[[1,0.5],[0.5,1]],[[1,-0.5],[-0.5,1]]])

    u = np.zeros(z.shape[0], dtype='float64')

    for i in range(m):
        u += multivariate_normal_np(z, means[i], covs[i], d)

    return u

def multivariate_normal(x, mean, cov, d):
    """pdf of the multivariate normal distribution."""
    # 2-dimension    
    x_m = (x - mean).T

    first_term = 1. / (torch.sqrt((2 * torch.pi)**d * torch.linalg.det(cov)))
    temp = torch.linalg.solve(cov, x_m)
    u = torch.matmul(temp.T, x_m) / 2
    u *= first_term

    return u.squeeze(-1)

    
def gaussian_mixture(z):
    # d : dimension of x - Multivariate Gaussian
    # m : number of mixtures - Gaussian Mixture
    # means : m x d
    d = 2
    m = 2
    means = torch.tensor([[-2,0.5],[0.5,-2],[1.5,1]], dtype=torch.double, requires_grad=True)
    covs = torch.tensor([[[0.5,0],[0,1]],[[0.5,0],[0,1]],[[1,0.5],[0.5,1]],[[1,-0.5],[-0.5,1]]], dtype=torch.double, requires_grad=True)

    z = z.view(-1, 2).double()

    u = 0
    for i in range(m):
        u += multivariate_normal(z, means[i], covs[i], d)

    return u

In [ ]:
r = np.linspace(-3, 3, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = gaussian_mixture_np(z)
q0 = np.array(q0)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Gaussian mixture')
plt.show()

In [ ]:
samples4 = FLMC.FLA(gaussian_mixture, alpha=1.75, N =200000, a_eta=1e-7, b_eta=0.6, step_size=0.001, dim=2)

plt.hist2d(samples4[:,0], samples4[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #4')
plt.show()